In [4]:
# functions

from scipy.integrate import odeint
from scipy.integrate import ode
import numpy as np
import matplotlib.pyplot as plt
import math
import pylab as pp
from scipy import integrate, interpolate
from scipy import optimize
import random
from collections import defaultdict

    
def birth(k0,k1,k2,d0,d1):
    return (k0 + d0)/(1 + k1/(k0 + d0) + (k1/(k0 + d0))*(k2/(k1 + d1)) + (k1/(k0 + d0))*(k2/(k1 + d1)))
    
    
    
def incr_rout_catch(t, start,coverage,eff):
    cov = [np.round(hyp(x,coverage,0.1),2) for x in range(1,10)]
    covc = [0.2,0.35,0.5,0.75,0.95]
    if t > start and t <= start + 365:
        return 1-(1-min(cov[0]+covc[0],1)*eff)**(1/365)
    if t > start + 365 and t <= start + 365*2:
        return 1-(1-min(cov[1]+covc[1],1)*eff)**(1/365)
    if t > start + 365*2 and t <= start + 365*3:
        return 1-(1-min(cov[2]+covc[2],1)*eff)**(1/365)
    if t > start + 365*3 and t <= start + 365*4:
        return 1-(1-min(cov[3]+covc[3],1)*eff)**(1/365)
    if t > start + 365*4 and t <= start + 365*5:
        return 1-(1-min(cov[4]+covc[4],1)*eff)**(1/365)
    if t > start + 365*5 and t <= start + 365*6:
        return 1-(1-cov[5]*eff)**(1/365)
    if t > start + 365*6 and t <= start + 365*7:
        return 1-(1-cov[6]*eff)**(1/365)
    if t > start + 365*7 and t <= start + 365*8:
        return 1-(1-cov[7]*eff)**(1/365)
    if t > start + 365*8 :
        return 1-(1-cov[8]*eff)**(1/365)
#         return coverage
    else:
        return 0


    
def routine(t, start,coverage):
    if t > start:
        return 1-(1-coverage)**(1/365)
#         return coverage/365
    else:
        return 0
    
def compute_average(ls,period):
    return [sum(ls[i:i+period])/period for i in range(0, len(ls), period)]

In [5]:
def all_countries_both(cov_rout,cov_cu):
    v3 = 0
    v3r = 0

    K = [  9.04006139e-03,   0.56856479e-01,   0.04761361e-06, 0.26080919e-03,   1.010820285e-01,   0.03052162e-02]


# manually set efficacy to desired level
    efficacy = 0.95
    coverage = cov_rout
    coverager= cov_cu


# compute death rates to keep population constant
    dS1n = (qn - agn1*K[0])/agn1
    dS2n = (K[0] * agn1 - K[1]*agn2)/agn2
    dS3n = (K[1] * agn2 - K[2]*agn3)/agn3
    dS4n = (K[2] * agn3)/agn4

    dS1c = (qc - agc1*K[3])/agc1
    dS2c = (K[3] * agc1 - K[4]*agc2)/agc2
    dS3c = (K[4] * agc2 - K[5]*agc3)/agc3
    dS4c = (K[5] * agc3)/agc4

    dI2n = (K[0] * agn1 - K[1]*agn2)/agn2
    dI3n = (K[1] * agn2 - K[2]*agn3)/agn3
    dI4n = (K[2] * agn3)/agn4 + di

    dI2c = (K[3] * agc1 - K[4]*agc2)/agc2
    dI3c = (K[4] * agc2 - K[5]*agc3)/agc3
    dI4c = (K[5] * agc3)/agc4 + di

    dR2n = (K[0] * agn1 - K[1]*agn2)/agn2
    dR3n = (K[1] * agn2 - K[2]*agn3)/agn3
    dR4n = (K[2] * agn3)/agn4

    dR2c = (K[3] * agc1 - K[4]*agc2)/agc2
    dR3c = (K[4] * agc2 - K[5]*agc3)/agc3
    dR4c = (K[5] * agc3)/agc4

    dC4n = (K[2] * agn3)/agn4
    dC4c = (K[5] * agc3)/agc4

    INPUT = np.array((s1n, s2n, s3n, s4n, s1c, s2c, s3c, s4c, 
                      i2n, i3n, i4n, i2c, i3c, i4c, 
                      r2n, r3n, r4n, r2c, r3c, r4c,
                      c4n, c4c, w,
                      vs1n, vs2n, vs3n, vs4n, vs1c, vs2c, vs3c, vs4c,
                      vi2n, vi3n, vi4n, vi2c, vi3cc, vi4cc, 
                      vr2n, vr3n, vr4n, vr2c, vr3c, vr4c,
                      cr4n, cr4c, cvs4n, cvs4c, cvr4n, cvr4c,0))



    def sys_equations(INP, t):

        V = INP
        Y = np.zeros((50))

        weather = (1 + forcing*np.cos(g*np.pi/365*(t-peak)))

        vt = 170

   
        v3 = routine(t,365*(vt+1), efficacy*cov_rout)   

# carriers
        carr = V[20] + V[21] + V[43] + V[44] + V[45] + V[46] + V[47] + V[48] 


     # susceptibles no comorbidity
        Y[0] = qn - (K[0] + dS1n)*V[0] - v1*V[0] + l1*V[23]
        Y[1] = K[0]*V[0] + phi*V[14] - ((beta2n*(b24*carr + weather*V[22])) + dS2n + K[1])*V[1] - v2*V[1] + l2*(V[24] + V[31] + V[37])
        Y[2] = K[1]*V[1] + phi*V[15] - ((beta3n*(b34*carr + weather*V[22])) + dS3n + K[2])*V[2] - v3*V[2] + l3*(V[25] + V[32] + V[38])
        Y[3] = K[2]*V[2] + phi*V[16] + psi4n*V[20] - ((beta4n*(b44*carr + weather*V[22])) + dS4n)*V[3]

    # susceptibles with comorbidity
        Y[4] = qc - (K[3] + dS1c)*V[4] - v1*V[4] + l1*V[27]
        Y[5] = K[3]*V[4] + phi*V[17] - ((beta2c*(b24*carr + weather*V[22])) + dS2c + K[4])*V[5] - v2*V[5] + l2*(V[28] + V[34] + V[40])
        Y[6] = K[4]*V[5] + phi*V[18] - ((beta3c*(b34*carr + weather*V[22])) + dS3c + K[5])*V[6] - v3*V[6] + l3*(V[29] + V[35] + V[41])
        Y[7] = K[5]*V[6] + phi*V[19] + psi4c*V[21] - ((beta4c*(b44*carr + weather*V[22])) + dS4c)*V[7]

    # infected without comorbidity
        Y[8] = (beta2n*(b24*carr + weather*V[22]))*V[1] - (rho2n + dI2n + K[1])*V[8] 
        Y[9] = K[1]*V[8] + (beta3n*(b34*carr + weather*V[22]))*V[2] - (rho3n + dI3n + K[2])*V[9] - v3*V[9] - dr*Y[9]
        Y[10] = K[2]*V[9] - (rho4n + dI4n)*V[10]

    # infected with comorbidity    
        Y[11] = (beta2c*(b24*carr+ weather*V[22]))*V[5] - (rho2c + dI2c + K[4])*V[11]
        Y[12] = K[4]*V[11] + (beta3c*(b34*carr + weather*V[22]))*V[6] - (rho3c + dI3c + K[5])*V[12] - v3*V[12] - drc*Y[12]
        Y[13] = K[5]*V[12] + (beta4c*(b44*carr + weather*V[22]))*alphac*V[7] + a4c*V[21] - (rho4c + dI4c)*V[13] - drc*Y[13] 

    #  recovered no comorbidity  
        Y[14] = rho2n*V[8] - (dR2n + K[1] +  phi)*V[14] 
        Y[15] = K[1]*V[14] + rho3n*V[9] - (dR3n + phi + K[2])*V[15] - v3*V[15]
        Y[16] = K[2]*V[15] + rho4n*V[10] + psi4n*V[43] - ((beta4n*(b44*carr + weather*V[22])) + dR4n + phi)*V[16]

    # recovered with comorbidity
        Y[17] = rho2c*V[11] - (dR2c + K[4] + phi)*V[17] 
        Y[18] = K[4]*V[17] + rho3c*V[12] - (dR3c + phi + K[5])*V[18] - v3*V[18]#*(1-v3) 
        Y[19] = K[5]*V[18] + rho4c*V[13] + psi4c*V[44] - ((beta4c*(b44*carr + weather*V[22]))+dR4c + phi)*V[19]

    # carrier no comorbidity
        Y[20] = (beta4n*(b44*carr + weather*V[22]))*((1-alphan)*V[3]) - (psi4n + dC4n)*V[20]

    # carrier with comorbidity
        Y[21] = 0

    #  carrier rec no com
        Y[43] = (beta4n*(b44*carr + weather*V[22]))*V[16] - (psi4n + dC4n)*V[43]

    # carrier rec with com
        Y[44] = (beta4c*(b44*carr + weather*V[22]))*V[19] - (psi4c + dC4c)*V[44]

    # carrier vsus no comorbidity
        Y[45] = (beta4n*(b44*carr + weather*V[22]))*V[26] - (psi4n + dC4n)*V[45] 

    # carrier vsus with comorbidity
        Y[46] = (beta4c*(b44*carr + weather*V[22]))*V[30] - (psi4c + dC4c)*V[46] 

    # carrier vrec no comorbidity
        Y[47] = (beta4n*(b44*carr + weather*V[22]))*V[39] - (psi4n + dC4n)*V[47] 

    # carrier vrec with comorbidity
        Y[48] = (beta4c*(b44*carr + weather*V[22]))*V[42] - (psi4c + dC4c)*V[48] 


    # water
        Y[22] = gumma*(carr) - csi*V[22]

    # vaccinated susceptibles no comorbidity
        Y[23] = v1*V[0] - (dS1n + K[0] + l1)*V[23]
        Y[24] = v2*V[1] + K[0]*V[23] - (dS2n + K[1] + l2)*V[24]
        Y[25] = v3*V[2] + K[1]*V[24] - (dS3n + K[2] + l3)*V[25]
        Y[26] = v4*V[3] + K[2]*V[25] + psi4n*V[45] - ((beta4n*(b44*carr + weather*V[22])) + dS4n + l4)*V[26]

    # vaccinated susceptibles with comorbidity
        Y[27] = v1*V[4] - (dS1c + K[3] + l1)*V[27]
        Y[28] = v2*V[5] + K[3]*V[27] - (dS2c + K[4] + l2)*V[28]
        Y[29] = v3*V[6] + K[4]*V[28] - (dS3c + K[5] + l3)*V[29]
        Y[30] = v4*V[7] + K[5]*V[29] + psi4c*V[46] - ((beta4c*(b44*carr + weather*V[22]))+dS4c + l4)*V[30]

    # vaccinated infected no comorbidity
        Y[31] = 0
        Y[32] = v3*V[9]  - (dI3n + K[2] + l3)*V[32] 
        Y[33] = K[2]*V[32] - (dI4n + l4)*V[33] 

    # vaccinated infected with comorbidity
        Y[34] = 0
        Y[35] = v3*V[12]  - (dI3c + K[3] + l3)*V[35] 
        Y[36] = K[3]*V[35] - (dI4c + l4)*V[36]


    #   vaccinated recovered no comorbidity
        Y[37] = v2*V[14] - (dR2n + K[1] + l2)*V[37]
        Y[38] = v3*V[15] + K[1]*V[37] - (dR3n + K[2] + l3)*V[38]
        Y[39] = v4*V[16] + K[2]*V[38] + psi4n*V[47] - ((beta4n*(b44*carr + weather*V[22])) + dR4n + l4)*V[39]


    #   vaccinated recovered with comorbidity
        Y[40] = v2*V[17] - (dR2c + K[4] + l2)*V[40]
        Y[41] = v3*V[18] + K[4]*V[40] - (dR3c + K[5] + l3)*V[41]
        Y[42] = v4*V[19] + K[5]*V[41] + psi4c*V[48] - ((beta4c*(b44*carr + weather*V[22])) + dR4c + l4)*V[42]

        Y[49] =  dr*Y[8] + dr*Y[9] + drc*Y[11] + drc*Y[12]

        

        return Y

    # Initial paramaters regrouped + time array 
    vacdate = 170
    totaltime = 200
    time = np.arange(0.0, 365*totaltime, 1)

    result1 = odeint(sys_equations, INPUT, time[:vacdate*365]).T

    # catch-up only, 1 year
    end1 = [i[-1] for i in result1]
    INPUTTA1 = [end1[0],
                end1[1],
                end1[2]*(1-coverager*efficacy),
                end1[3],
                end1[4],
                end1[5],
                end1[6]*(1-coverager*efficacy),
                end1[7],
                end1[8],
                end1[9]*(1-coverager*efficacy),
                end1[10],
                end1[11],
                end1[12]*(1-coverager*efficacy),
                end1[13],
                end1[14],
                end1[15]*(1-coverager*efficacy),
                end1[16],
                end1[17],
                end1[18]*(1-coverager*efficacy),
                end1[19],
                end1[20],
                end1[21],
                end1[22],
                end1[23],
                end1[24],
                end1[2]*(coverager*efficacy),
                end1[26],
                end1[27],
                end1[28],
                end1[6]*(coverager*efficacy),
                end1[30],
                end1[31],
                end1[9]*(coverager*efficacy),
                end1[33],
                end1[34],
                end1[12]*(coverager*efficacy),
                end1[36],
                end1[37],
                end1[15]*(coverager*efficacy),
                end1[39],
                end1[40],
                end1[18]*(coverager*efficacy),
                end1[42],
                end1[43],
                end1[44],
                end1[45],
                end1[46],
                end1[47],
                end1[48],
                end1[49]]
    INPUT1 = np.array(INPUTTA1)

    result2 = odeint(sys_equations, INPUT1, time[vacdate*365:]).T


    allresults = [np.append(result1[i],result2[i]) for i in range(len(result1))]

    mxstep=500000

    return allresults



In [6]:
import pandas as pd
import pickle

covdf = pd.read_excel('CoverageDTP1.xlsx')

from collections import defaultdict
coverage_ = defaultdict(list)


for index,row in covdf.iterrows():
    coverage_[row['Country']].append((float(row['Coverage - dtp1'])/100,float(row['C-u coverage'])/100))
    
coverage_['SUB-SAHARAN AFRICA'] = [(0.92, 0.9)]
    
pyrs = pickle.load(open('pyramids_comorbidities_June21_medium.pkl','rb'))

In [7]:
# coverage of at least 90%
coverage_90 = defaultdict(list,
            {'Angola': [(0.9, 0.9)],
             'Benin': [(0.9, 0.9)],
             'Botswana': [(0.93, 0.93)],
             'Burkina Faso': [(0.9, 0.9)],
             'Burundi': [(0.92, 0.92)],
             'Cabo Verde': [(0.98, 0.98)],
             'Cameroon': [(0.9, 0.9)],
             'Central African Republic': [(0.9, 0.9)],
             'Chad': [(0.9, 0.9)],
             'Comoros': [(0.9, 0.9)],
             'Congo': [(0.9, 0.9)],
             "Côte d'Ivoire": [(0.92, 0.92)],
             'Democratic Republic of the Congo': [(0.92, 0.92)],
             'Equatorial Guinea': [(0.9, 0.9)],
             'Eritrea': [(0.93, 0.93)],
             'Eswatini': [(0.9, 0.9)],
             'Ethiopia': [(0.91, 0.91)],
             'Gabon': [(0.9, 0.9)],
             'Gambia': [(0.9, 0.9)],
             'Ghana': [(0.92, 0.92)],
             'Guinea': [(0.9, 0.9)],
             'Guinea-Bissau': [(0.9, 0.9)],
             'Kenya': [(0.9, 0.9)],
             'Lesotho': [(0.995, 0.995)],
             'Liberia': [(0.9, 0.9)],
             'Madagascar': [(0.92, 0.92)],
             'Malawi': [(0.92, 0.92)],
             'Mali': [(0.9, 0.9)],
             'Mauritania': [(0.9, 0.9)],
             'Mauritius': [(0.99, 0.99)],
             'Mozambique': [(0.95, 0.95)],
             'Namibia': [(0.9, 0.9)],
             'Niger': [(0.9, 0.9)],
             'Nigeria': [(0.9, 0.9)],
             'Rwanda': [(0.96, 0.96)],
             'Sao Tome and Principe': [(0.95, 0.95)],
             'Senegal': [(0.96, 0.96)],
             'Seychelles': [(0.99, 0.99)],
             'Sierra Leone': [(0.93, 0.93)],
             'South Africa': [(0.9, 0.9)],
             'Togo': [(0.9, 0.9)],
             'Uganda': [(0.9, 0.9)],
             'United Republic of Tanzania': [(0.99, 0.99)],
             'Zambia': [(0.93, 0.93)],
             'Zimbabwe': [(0.9, 0.9)],
             'Djibouti': [(0.93, 0.93)],
             'Mayotte': [(0.9, 0.9)],
             'Réunion': [(0.99, 0.99)],
             'Somalia': [(0.9, 0.9)],
             'South Sudan': [(0.9, 0.9)],
             'SUB-SAHARAN AFRICA': [(0.92, 0.92)]})

In [13]:
diff_ages = defaultdict(list)


for country, py in pyrs.items():



        pop1 = py[0][0][0]
        pop2 = py[0][0][1]
        pop3 = py[0][0][2]
        pop4 = py[0][0][3]
        popnumb = py[0][1]*1000

    
        if py[1][0] + py[1][1] > 0:
            bc = py[1][0] + py[1][1]
        else:
            bc = 0.000000000000001

        bn = 1 - bc

        q = 0.0363/365
        qn = q*bn
        qc = q*bc
        adHIV = py[1][2]


        # Weather/environment1
        # rate of shedding into water supply
        gumma = 1/7 #assumption
        forcing = 0.1
        g = 2.1
        peak = 30*8
        w = 1  
        weat = 1



        # contact matrix (none used for the paper, it can be inputed here)
        b24 = 1
        b34 = 1
        b44 = 1
        thetaR = 1
        thetaU = 1


        beta2n = 0.00022
        beta2c = beta2n*3.5
        beta3n = 0.00022
        beta3c = beta3n*3.5
        beta4n = 0.0082
        beta4c = beta4n*3.5

        alphan = 0.00
        alphac = 1



        # recovery rates
        rho = 1/14 
        rho2n = rho 
        rho3n = rho
        rho4n = rho
        rho2c = rho
        rho3c = rho
        rho4c = rho

        # waning rate of recovered immunity
        phi = 1/(365*7)

        # waning rate of bacteria in water
        csi = 1/50

        # carrier recovery rate
        psi4n = 1/20
        psi4c = psi4n

        # prob carrier becomes infected
        a4n = 0.
        a4c = 0.

        dr = 0.2
        drc = 0.2

        v1 = 0 
        v2 = 0
        v3 = 0
        v4 = 0
        v3r = 0
        
# vaccine immunity waning
        l1 = 0
        l2 = 0
        l3 = 0
        l4 = 0

        # intitial conditions

        s1n = pop1*bn
        s2n = pop2*bn
        s3n = pop3*bn 
        s4n = pop4*(1-adHIV)

        s1c = pop1*bc
        s2c = pop2*bc
        s3c = pop3*bc 
        s4c = 1-s1n-s2n-s3n-s4n-s1c-s2c-s3c

        i2n = 0.0
        i3n = 0.0
        i4n = 0
        i2c = 0.0
        i3c = 0.0
        i4c = 0

        r2n = 0.0
        r3n = 0.0
        r4n = 0
        r2c = 0.0
        r3c = 0.0
        r4c = 0.0

        c4n = 0.0
        c4c = 0.0

        vs1n = 0
        vs2n = 0
        vs3n = 0
        vs4n = 0
        vs1c = 0
        vs2c = 0 
        vs3c = 0 
        vs4c = 0
        vi2n = 0 
        vi3n = 0 
        vi4n = 0
        vi2c = 0 
        vi3cc = 0 
        vi4cc = 0
        vr2n = 0 
        vr3n = 0
        vr4n = 0
        vr2c = 0 
        vr3c = 0 
        vr4c = 0
        cr4n = 0
        cr4c = 0
        vc4n = 0
        vc4c = 0
        cvs4n = 0
        cvs4c = 0
        cvr4n = 0
        cvr4c = 0 

    

        age1 = s1n + s1c + vs1c + vs1n
        age2 = s2n + s2c + i2n + i2c + r2n + r2c + vs2c + vs2n + vi2c + vi2n + vr2c + vr2n
        age3 = s3n + s3c + i3n + i3c + r3n + r3c + vs3c + vs3n + vi3cc + vi3n + vr3c + vr3n
        age4 = s4n + s4c + i4n + i4c + r4n + r4c + c4n + c4c + vs4c + vs4n + vi4cc + vi4n + vr4c + vr4n + vc4c + vc4n


        agn1 = s1n + vs1n
        agn2 = s2n + i2n + r2n + vs2n + vi2n + vr2n
        agn3 = s3n + i3n + r3n + vs3n + vi3n + vr3n
        agn4 = s4n + i4n + r4n +  c4n + vs4n + vi4n + vr4n + vc4n
        agn4c = s4n + r4n + vs4n + vr4n

        agc1 = s1c + vs1c
        agc2 = s2c + i2c + r2c + vs2c + vi2c + vr2n
        agc3 = s3c + i3c + r3c + vs3c + vi3cc + vr3c
        agc4 = s4c + i4c + r4c +  c4c + vs4c + vi4cc + vr4c + vc4c
        agc4c = s4c + r4c + vs4c + vr4c

        nchild = agn1 + agn2 + agn3
        cchild = agc1 + agc2 + agc3
        
# to compute each scenario, just uncomment the corresponding line

#         no vaccine
#         diff_ages[country] = all_countries_both(0,0)
#         only catch-up
#         diff_ages[country] = all_countries_both(0,coverage_[country][0][1])
    #     only routine
#         diff_ages[country] = all_countries_both(coverage_[country][0][0],0)
    #    routine + catch-up
#         diff_ages[country] = all_countries_both(coverage_[country][0][0],coverage_[country][0][1])


In [15]:
# save the simulation output using pickle
# pickle.dump(diff_ages,open('country_specific_no_vacc.pkl','wb'))
# pickle.dump(diff_ages,open('country_specific_only_cu85.pkl','wb'))
# pickle.dump(diff_ages,open('country_specific_only_cu95_.pkl','wb'))
# pickle.dump(diff_ages,open('country_specific_only_routine85.pkl','wb'))
# pickle.dump(diff_ages,open('country_specific_only_routine95.pkl','wb'))
# pickle.dump(diff_ages,open('country_specific_cu+routine95.pkl','wb'))
# pickle.dump(diff_ages,open('country_specific_cu+routine85.pkl','wb'))